# Multi-modal RAG using Claude 3 Sonnet provided by Amazon Bedrock
Most of documents especially in finance domain, have a blend of content formats, such as textual data and visual representations such as bar charts and line graphs (images).

However, the information encapsulated within images is often disregarded in the majority of Retrieval-Augmented Generation (RAG) applications.

With the advent of multimodal Large Language Models (LLMs), like `Anthropic Claude 3 Sonnet`, it becomes relevant to explore methodologies for leveraging visual data in RAG applications.

## Steps followed in the notebook
1. Use a multimodal large language model (LLM), like Claude 3 Sonnet, to generate concise text descriptions from visual inputs (images).

2. Convert the textual summaries and raw image data into numerical representations (embeddings) and store them in a retrievable format, maintaining a linkage between the summaries and their corresponding images.

3. Feed the raw image data and relevant text chunks into a multimodal LLM, which can jointly process and synthesize information from both modalities to produce an answer.

We will use `pypdf` to parse images, and text from documents (PDFs).
We will use the [`multi-vector retriever`](#https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector) with Chroma DB to store raw text, tables and images along with their summaries for retrieval.

Reference:
[Multi-modal RAG](#https://github.com/sudarshan-koirala/youtube-stuffs/blob/main/langchain/LangChain_Multi_modal_RAG.ipynb) authored by Sudarshan Koirala

## Setup
Install required packages and import the libraries, which will be used throughout the notebook.
Make sure to restart the kernel after installing the packages.

In [ ]:
%pip install pypdf
%pip install matplotlib scikit-image
%pip install ftfy regex tqdm
%pip install chromadb
%pip install sqlalchemy==2.0.0
%pip install --upgrade langchain>=0.1.0

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

## Download data

In [ ]:
!rm -r images
!mkdir images

In [ ]:
import urllib.request

url = "https://sgp.fas.org/crs/misc/IF10244.pdf"
filename = "wildfire_stats.pdf"
urllib.request.urlretrieve(url,filename)

## Extract images and text from the pdf

In [ ]:
path = "images"

In [ ]:
import langchain
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter
from pypdf import PdfReader

reader = PdfReader(f"{filename}")
print('# of pages: ', len(reader.pages))
image_count = 0
texts = []

# extract and save images
for page in reader.pages:
    for image in page.images:
        image_count+=1
        with open(f"{path}/{image.name}", 'wb') as f:
            f.write(image.data)

loader = PyPDFLoader(filename)
documents = loader.load_and_split()

text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)

docs = text_splitter.split_documents(documents)
print("# of text chunks: ", len(docs))
print("# of images in document: ", image_count)

### Plot the images in the file

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

image_paths = []
for img_path in os.listdir("./images"):
    image_paths.append(str(os.path.join("./images", img_path)))


def plot_images(image_paths):
    images_shown = 0
    plt.figure(figsize=(16, 9))
    for img_path in image_paths:
        if ".png" in img_path:
            image = Image.open(img_path)

            plt.subplot(3, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])
            images_shown += 1
            if images_shown >= 9:
                break

print("Images in the document")
plot_images(image_paths)


## Create image summaries using Claude 3 Sonnet
We will use Claude 3 Sonnet model from Anthropic to create image summaries, which we will store in the `multi-vector retriever` from Langchain.

Before that, lets first set up a connection to the Anthropic Claude multimodal model using the AWS Bedrock service. Imports necessary libraries, configure the AWS Boto3 client, and initialize the Bedrock chat model interface using the LangChain library. 

In [ ]:
import boto3
import pprint
from botocore.client import Config
import json
from langchain_community.chat_models.bedrock import BedrockChat

pp = pprint.PrettyPrinter(indent=2)
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' # change this to use a different version from the model provider
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
print(region)
accept = 'application/json'
contentType = 'application/json'

Now we will define two functions: `encode_image` and `invoke_model_sonnet`. 
- The `encode_image` function takes an image file path as input and returns the base64 encoded string representation of the image file. 
- The `invoke_model_sonnet` function takes a base64 encoded image and a prompt as input, initializes a `BedrockChat` instance, creates a message list with the prompt and image, invokes the chat model, and returns the model's response.

In [ ]:
import io
import os
import base64
import numpy as np
from PIL import Image
from langchain.chat_models.bedrock import BedrockChat
from langchain.schema.messages import HumanMessage, SystemMessage

def encode_image(image_path):
    ''' Getting the base64 string '''
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def invoke_model_sonnet(img_base64,prompt):
    ''' Image summary '''
    chat = BedrockChat(model_id=modelId,
                      client=bedrock_client,
                      model_kwargs={
                      'temperature': 0.1, 
                      'top_p': 0.1
                  } 
                      )
    content = [
        {"type": "text", "text": prompt},
        {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": img_base64}}
        ]
    message_list = [
    {
        "role": 'user',
        "content": content
    }
]
    msg = chat.invoke(message_list)
    return msg.content

In [ ]:

# Prompt
prompt = """
Summarize all the details in the image, focus on the statistics provided in the image such as bar charts and graphs. 
Make sure to include the time period in the summary, if available. For example, the images shows line graph showing data from 1993 to 2022.
"""

# Read images, encode to base64 strings
def create_image_summaries(input_files):
    img_base64_list = []
    image_summaries = []
    for file in input_files:
        print(file)
        base64_image = encode_image(file)
        image_summaries.append(invoke_model_sonnet(base64_image,prompt))
        img_base64_list.append(base64_image)
    return img_base64_list, image_summaries
img_base64_list, image_summaries = create_image_summaries(image_paths)



In [ ]:
# lets view the summary of one of the images
from IPython.display import display, HTML

def plt_img_base64(img_base64):

    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'

    # Display the image by rendering the HTML
    display(HTML(image_html))

plt_img_base64(img_base64_list[3])
print("------------------ Image Summary ---------------\n", image_summaries[3])

## Add data into the Vector Store

Now we will set up the necessary components for a [multi-vector retriever](#https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector) system using the Langchain library. We will import required modules, create an instance of the BedrockEmbeddings for text embeddings, initialize a Chroma vector store for indexing document chunks, and set up an in-memory store for parent documents. Finally, we will create a `MultiVectorRetriever` object, which combines the vector store and document store for efficient document retrieval based on vector embeddings.

In [ ]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.embeddings import BedrockEmbeddings

embedding_model = f'cohere.embed-english-v3'
# instantiate embeddings model.
embedding_model = BedrockEmbeddings(
    client=bedrock_client,
    model_id=embedding_model
)
# define the vector store
vectorstore = Chroma(collection_name="multi_modal_rag",
                     embedding_function=embedding_model)

# define the storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# the retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
)

In [ ]:
# Add texts
doc_ids = [str(uuid.uuid4()) for _ in docs]
texts = []
for i, doc in enumerate(docs):
    doc.metadata.update({id_key: doc_ids[i]})
    texts.append(doc.page_content)
retriever.vectorstore.add_documents(docs)
retriever.docstore.mset(list(zip(doc_ids, texts)))

In [ ]:
# Add image summaries
img_ids = [str(uuid.uuid4()) for _ in img_base64_list]
summary_img = [
    Document(page_content=s, metadata={id_key: img_ids[i]})
    for i, s in enumerate(image_summaries)
]
retriever.vectorstore.add_documents(summary_img)
retriever.docstore.mset(list(zip(img_ids, img_base64_list)))

### Verify the retrieved data
Now that we have added our text, image summaries and base64 images to the vector store, let's check that the retrieval is giving the relevant documents related to the query by following below steps: 
- We will first retrieve relevant documents using a retriever object and  write the helper function to split the retrieved documents into base64-encoded images and text chunks. 
- Finally print the number of retrieved images and text chunks. We will verify the result by displaying the first retrieved image, and printing the dictionary containing the separated images and text chunks.

In [ ]:
docs = retriever.get_relevant_documents(
    "What is the change in wild fires from 1993 to 2022?"
)
len(docs)

In [ ]:
from base64 import b64decode
def split_image_text_types(docs):
    ''' Split base64-encoded images and texts '''
    b64 = []
    text = []
    for doc in docs:
        try:
            b64decode(doc)
            b64.append(doc)
        except Exception as e:
            text.append(doc)
    return {
        "images": b64,
        "texts": text
    }
docs_by_type = split_image_text_types(docs)
print("# images retrieved: ", len(docs_by_type["images"]))
print("# of text chunk retrieved: ", len(docs_by_type["texts"]))
print('---------------- Retrieved Image -------------')
if docs_by_type["images"]:
    plt_img_base64(docs_by_type["images"][0])

In [ ]:

pprint.pp(docs_by_type["texts"][1])


In [ ]:
docs_by_type

## Retrieval Augmented Generation workflow
Now that we have verified our retrievals, lets test our solution end to end.
- We will first define a function `prompt_func` that takes a dictionary as input and returns a list of messages formatted for a `Anthropic Claude 3 Sonnet` model. 
    - The function concatenates the text content from the input dictionary and formats it along with the `question` as a prompt for the model. It also includes any images from the input dictionary as base64-encoded data in the prompt. 
- Next, we will set up a pipeline using the LangChain library, to perform retrieval, followed by passing the retrieved data to the `prompt_func`, which sends the formatted prompt to `Claude 3 Sonnet model`, finally parsing the output using a string output parser (`StrOutputParser`). 
- We will then invoke the pipeline with the question to get the generated answer.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.schema.output_parser import StrOutputParser

def prompt_func(dict):
    format_texts = "\n".join(dict["context"]["texts"])
    prompt = f'''Answer the question based only on the following context, which can include text and images:
    Question: {dict["question"]}
    Text:
    {format_texts}
    '''
    content = [{"type": "text", "text": prompt}]
    
    for image in dict["context"]["images"]: 
        content.append(
            {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": image}},
        )
    # pprint.pp(content)
    message_list = [
    {
        "role": 'user',
        "content": content
    }
]
    
    return message_list

chat_model = BedrockChat(model_id=modelId,
                      client=bedrock_client,
                       model_kwargs={
                      'temperature': 0.1, 
                      'top_p': 0.1
                  } )
# RAG pipeline
chain = (
    {"context": retriever | RunnableLambda(split_image_text_types), "question": RunnablePassthrough()}
    | RunnableLambda(prompt_func)
    | chat_model
    | StrOutputParser()
)

In [ ]:
generated_answer = chain.invoke(
    "What is the change in wild fires from 1993 to 2022?"
)
pprint.pp(generated_answer)

Please note that the above response uses both the retrieved text and images, to generate the final answer. 

## Conclusion
In this notebook, we provided the technique to build RAG applications which understand both text and visual data encapsulated in images to provide more relevant and accurate responses to the user. In this approach, we specifically leveraged multimodal Large Language Model (LLM), `Claude 3 Sonnet`, for leveraging both text and visual data for building our RAG application.